In [1]:
import duckdb
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Dataset connection
df = pd.read_csv('/Users/jinlinchen/Documents/Study/HWR Berlin/Semester 2/Analytics Lab/Analytics Project/Dataset part/preprocessed_data.csv')

In [3]:
# Database connection
# Open database can check your connection settings (Host --> path) 
# make sure to close your dbeaver or error might occur, or make a copy of your database
db_connection_string = '/Users/jinlinchen/Documents/Study/HWR Berlin/Semester 2/Analytics Lab/Data Resource/isrecon_AIS11.duckdb'
conn = duckdb.connect(db_connection_string)

In [4]:
# List all tables in the database
tables = conn.execute("SHOW TABLES").fetchdf()
print(tables)

            name
0        authors
1      citations
2       entities
3   entity_count
4       keywords
5       ontology
6         papers
7     paragraphs
8      sentences
9        sources
10   subsections
11      synonyms


In [6]:
# Query the database to join entities and paragraphs on para_id
query = """
SELECT e.entity, e.sentence, p.paragraph, s.sentence_original
FROM entities e
JOIN paragraphs p ON e.para_id = p.para_id
JOIN sentences s ON e.sentence_id = s.sentence_id
"""
db_data = conn.execute(query).fetchdf()

In [25]:
# Check database data
db_data

,entity,sentence,paragraph,sentence_original
0,product innovation,A B S T R A C TFirm - sponsored online user co...,Abstract. Firm-sponsored online user communiti...,Abstract. Firm-sponsored online user communiti...
1,decision-making,In addition to potentially improving customer ...,Firm-hosted online user communities (hereinaft...,In addition to potentially improving customer ...
2,business strategy,"Firms across industries , especially in the hi...",Firm-hosted online user communities (hereinaft...,"Firms across industries, especially in the hig..."
3,platform strategy,"SAP , for example , has implemented its OUC ( ...",Firm-hosted online user communities (hereinaft...,"SAP, for example, has implemented its OUC (SAP..."
4,Salesforce.com,Examples of other well - known OUC include Sal...,Firm-hosted online user communities (hereinaft...,Examples of other well-known OUC include Sales...
...,...,...,...,...
4654131,privacy,"According to Varian , people agree to the ' in...","The last stage of the laboring society, the so...","According to Varian, people agree to the 'inva..."
4654132,IT management,"Comparing IT governance and IT management , ST...",Preposition 2: There may be a deficit in those...,"Comparing IT governance and IT management, Pet..."
4654133,chisquare = 588.36,The test produced poor model fit statistics ( ...,"In terms of ex post approaches, we ran Harman'...",The test produced poor model fit statistics (c...
4654134,IT governance,"Comparing IT governance and IT management , ST...",Preposition 2: There may be a deficit in those...,"Comparing IT governance and IT management, Pet..."


In [8]:
#check initial dataset
df.head()

,ID,entity,ent_id,sentence_original,class_ID,class_name,sentence_id,sentence_preprocessed,sentence_stemmed,sentence_lemmas,sentence_tokenized
0,90812,interviews,qualitative interview,Especially for the problem of reduced identity...,1,belongs_to_article,507_6563_6594,especially problem reduced identity efforts vt...,especi problem reduc ident effort vt identifi ...,especially problem reduce identity effort vt i...,"['especially', 'problem', 'reduced', 'identity..."
1,90770,knowledge management,knowledge management,This makes it challenging for them to organize...,3,background_information,507_4787_4813,makes challenging organize meetings conversati...,make challeng organ meet convers well perform ...,make challenge organize meeting conversation w...,"['makes', 'challenging', 'organize', 'meetings..."
2,90500,reference model,reference modelling,It can be foreseen that managing such a compre...,3,background_information,505_7541_7567,foreseen managing comprehensive reference mode...,foreseen manag comprehens refer model challeng...,foresee manage comprehensive reference model c...,"['fore', '##see', '##n', 'managing', 'comprehe..."
3,90417,UML,unified modeling language,"In particular, redefinition in UML allows one ...",3,background_information,505_4085_4121,particular redefinition uml allows one modify ...,particular redefinit uml allow on modifi data ...,particular redefinition uml allow one modify d...,"['particular', 'red', '##ef', '##ini', '##tion..."
4,90361,reference modeling,reference modelling,"At the same time, redundancy in a reference mo...",2,related_work,505_2464_2508,time redundancy reference model typically avoi...,time redund refer model typic avoid see e g co...,time redundancy reference model typically avoi...,"['time', 'red', '##unda', '##ncy', 'reference'..."


In [31]:
# Filter out the duplicates, because we want to have examples where the same sentence is annotated with different entities.
duplicates = db_data[db_data.duplicated(subset=['sentence_original'], keep=False)]
duplicates_with_diff_entities = duplicates[duplicates.duplicated(subset=['sentence_original'], keep=False)]

In [32]:
# Descending order on column 'sentence_original'
duplicates_with_diff_entities = duplicates_with_diff_entities.sort_values(by='sentence_original', ascending=False)

In [33]:
# Check the result
duplicates_with_diff_entities

,entity,sentence,paragraph,sentence_original
340820,algorithms,➢ Using the data warehouse as an infrastructur...,➢ Coupling the data warehousing approaches and...,➢ Using the data warehouse as an infrastructur...
342460,data warehouse,➢ Using the data warehouse as an infrastructur...,➢ Coupling the data warehousing approaches and...,➢ Using the data warehouse as an infrastructur...
2786156,algorithm,➢ Logic errors are incorrect formulas due to c...,"➢ Mechanical errors are typing errors, pointin...",➢ Logic errors are incorrect formulas due to c...
2787155,algorithm,➢ Logic errors are incorrect formulas due to c...,"➢ Mechanical errors are typing errors, pointin...",➢ Logic errors are incorrect formulas due to c...
3042165,Hypothesis,✗ END_PARAGRAPH_TAG START_PARAGRAPH_TAG Hypoth...,"✗ ✓ Hypothesis supported, (✓) Hypothesis parti...","✗ ✓ Hypothesis supported, (✓) Hypothesis parti..."
...,...,...,...,...
375218,theories, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
370084,information systems research, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
456931,theories, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
4603673,IT, ...,Five established IT consulting firms were ...,Five established IT consulting firms were ...


In [34]:
# Remove duplicates where the sentence does not start with a letter
import string
def starts_with_letter(sentence):
    return sentence.strip()[0].lower() in string.ascii_lowercase
filtered_duplicates = duplicates_with_diff_entities[duplicates_with_diff_entities['sentence_original'].apply(starts_with_letter)]


In [35]:
#check
filtered_duplicates

,entity,sentence,paragraph,sentence_original
681178,mean of 500,zy refers to the mean of 500 path estimates fr...,xy refers to the mean of 500 path estimates fr...,zy refers to the mean of 500 path estimates fr...
685069,mean of 500,zy refers to the mean of 500 path estimates fr...,xy refers to the mean of 500 path estimates fr...,zy refers to the mean of 500 path estimates fr...
678387,moderator variable,zy refers to the mean of 500 path estimates fr...,xy refers to the mean of 500 path estimates fr...,zy refers to the mean of 500 path estimates fr...
3919500,web services,zur Muehlen et al . examined the development o...,. zur Muehlen et al. examined the development ...,zur Muehlen et al. examined the development of...
3920064,process management,zur Muehlen et al . examined the development o...,. zur Muehlen et al. examined the development ...,zur Muehlen et al. examined the development of...
...,...,...,...,...
375218,theories, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
370084,information systems research, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
456931,theories, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
4603673,IT, ...,Five established IT consulting firms were ...,Five established IT consulting firms were ...


In [36]:
# Filter out the duplicates, because we want to have examples where the same sentence is annotated with different entities.
filtered_duplicates = filtered_duplicates.drop_duplicates(subset=['entity', 'sentence_original'])

In [37]:
# check result
filtered_duplicates

,entity,sentence,paragraph,sentence_original
681178,mean of 500,zy refers to the mean of 500 path estimates fr...,xy refers to the mean of 500 path estimates fr...,zy refers to the mean of 500 path estimates fr...
678387,moderator variable,zy refers to the mean of 500 path estimates fr...,xy refers to the mean of 500 path estimates fr...,zy refers to the mean of 500 path estimates fr...
3919500,web services,zur Muehlen et al . examined the development o...,. zur Muehlen et al. examined the development ...,zur Muehlen et al. examined the development of...
3920064,process management,zur Muehlen et al . examined the development o...,. zur Muehlen et al. examined the development ...,zur Muehlen et al. examined the development of...
67939,CIO,zooming in on the two directions of understand...,zooming in on the two directions of understand...,zooming in on the two directions of understand...
...,...,...,...,...
377216,knowledge base, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
375218,theories, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
370084,information systems research, Add theories for e - government to the knowl...,Perform in-depth discussions and analyses of ...,Add theories for e-government to the knowledg...
4603673,IT, ...,Five established IT consulting firms were ...,Five established IT consulting firms were ...


In [38]:
# Save the first 1000 examples
filtered_1000 = filtered_duplicates.head(1000)

In [39]:
# check result
filtered_1000

,entity,sentence,paragraph,sentence_original
681178,mean of 500,zy refers to the mean of 500 path estimates fr...,xy refers to the mean of 500 path estimates fr...,zy refers to the mean of 500 path estimates fr...
678387,moderator variable,zy refers to the mean of 500 path estimates fr...,xy refers to the mean of 500 path estimates fr...,zy refers to the mean of 500 path estimates fr...
3919500,web services,zur Muehlen et al . examined the development o...,. zur Muehlen et al. examined the development ...,zur Muehlen et al. examined the development of...
3920064,process management,zur Muehlen et al . examined the development o...,. zur Muehlen et al. examined the development ...,zur Muehlen et al. examined the development of...
67939,CIO,zooming in on the two directions of understand...,zooming in on the two directions of understand...,zooming in on the two directions of understand...
...,...,...,...,...
2320388,app,"where β 01 and β 02 are the fixed intercepts ,...","where β 01 and β 02 are the fixed intercepts, ...","where β 01 and β 02 are the fixed intercepts, ..."
2148129,principal component analysis,"where α , β , γ , and δ are different weights ...","where α, β, γ, and δ are different weights of ...","where α, β, γ, and δ are different weights of ..."
2146735,PCA,"where α , β , γ , and δ are different weights ...","where α, β, γ, and δ are different weights of ...","where α, β, γ, and δ are different weights of ..."
1964449,mean value,where α j denotes the shared frailty of consum...,where α j denotes the shared frailty of consum...,where α j denotes the shared frailty of consum...


In [40]:
# Save the first 1000 examples to a csv file for manual labeling. 
csv_file_path = '/Users/jinlinchen/Documents/Study/HWR Berlin/Semester 2/Analytics Lab/Analytics Project/Database Part/manual_check.csv'
filtered_1000.to_csv(csv_file_path, index=False)

250 are labeled by each team member.

In [ ]:
# Load the manually labeled data
df_manual = pd.read_csv('/Users/jinlinchen/Documents/Study/HWR Berlin/Semester 2/Analytics Lab/Analytics Project/Database Part/manual_label - consolidated.csv')